**Reinforcement Learning with Q Learning**
* This notebook shows how to apply the classic Reinforcement Learning (RL) idea of Q learning 
* In TD learning we estimated state values: V(s). In Q learning we estimate Q values: Q(s,a). Here we'll go over Q learning in the simple tabular case.
* A key concept in RL is exploration. We'll use epsilon greedy exploration, which is often used with Q learning.

Outline:
1. Define the GridWorld environment
2. Discuss Epsilon-Greedy Exploration
3. Find the value of each Q value in the environment using Q learning





**GridWorld**

The GridWorld environment is a four by four grid. The agent randomly starts on the grid and can move either up, left, right, or down. If the agent reaches the upper left or lower right the episode is over. Every action the agent takes gets a reward of -1 until you reach the upper left or over right.

In [0]:
#Environment from: https://github.com/dennybritz/reinforcement-learning/blob/cee9e78652f8ce98d6079282daf20680e5e17c6a/lib/envs/gridworld.py

#define the environment

import io
import numpy as np
import sys
from gym.envs.toy_text import discrete
import pprint

UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

class GridworldEnv(discrete.DiscreteEnv):
    """
    Grid World environment from Sutton's Reinforcement Learning book chapter 4.
    You are an agent on an MxN grid and your goal is to reach the terminal
    state at the top left or the bottom right corner.
    For example, a 4x4 grid looks as follows:
    T  o  o  o
    o  x  o  o
    o  o  o  o
    o  o  o  T
    x is your position and T are the two terminal states.
    You can take actions in each direction (UP=0, RIGHT=1, DOWN=2, LEFT=3).
    Actions going off the edge leave you in your current state.
    You receive a reward of -1 at each step until you reach a terminal state.
    """

    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self, shape=[4,4]):
        if not isinstance(shape, (list, tuple)) or not len(shape) == 2:
            raise ValueError('shape argument must be a list/tuple of length 2')

        self.shape = shape

        nS = np.prod(shape)
        nA = 4

        MAX_Y = shape[0]
        MAX_X = shape[1]

        P = {}
        grid = np.arange(nS).reshape(shape)
        it = np.nditer(grid, flags=['multi_index'])

        while not it.finished:
            s = it.iterindex
            y, x = it.multi_index

            # P[s][a] = (prob, next_state, reward, is_done)
            P[s] = {a : [] for a in range(nA)}

            is_done = lambda s: s == 0 or s == (nS - 1)
            reward = 0.0 if is_done(s) else -1.0
            #reward = 1.0 if is_done(s) else 0.0

            # We're stuck in a terminal state
            if is_done(s):
                P[s][UP] = [(1.0, s, reward, True)]
                P[s][RIGHT] = [(1.0, s, reward, True)]
                P[s][DOWN] = [(1.0, s, reward, True)]
                P[s][LEFT] = [(1.0, s, reward, True)]
            # Not a terminal state
            else:
                ns_up = s if y == 0 else s - MAX_X
                ns_right = s if x == (MAX_X - 1) else s + 1
                ns_down = s if y == (MAX_Y - 1) else s + MAX_X
                ns_left = s if x == 0 else s - 1
                P[s][UP] = [(1.0, ns_up, reward, is_done(ns_up))]
                P[s][RIGHT] = [(1.0, ns_right, reward, is_done(ns_right))]
                P[s][DOWN] = [(1.0, ns_down, reward, is_done(ns_down))]
                P[s][LEFT] = [(1.0, ns_left, reward, is_done(ns_left))]

            it.iternext()

        # Initial state distribution is uniform
        isd = np.ones(nS) / nS

        # We expose the model of the environment for educational purposes
        # This should not be used in any model-free learning algorithm
        self.P = P

        super(GridworldEnv, self).__init__(nS, nA, P, isd)

    def _render(self, mode='human', close=False):
        """ Renders the current gridworld layout
         For example, a 4x4 grid with the mode="human" looks like:
            T  o  o  o
            o  x  o  o
            o  o  o  o
            o  o  o  T
        where x is your position and T are the two terminal states.
        """
        if close:
            return

        outfile = io.StringIO() if mode == 'ansi' else sys.stdout

        grid = np.arange(self.nS).reshape(self.shape)
        it = np.nditer(grid, flags=['multi_index'])
        while not it.finished:
            s = it.iterindex
            y, x = it.multi_index

            if self.s == s:
                output = " x "
            elif s == 0 or s == self.nS - 1:
                output = " T "
            else:
                output = " o "

            if x == 0:
                output = output.lstrip()
            if x == self.shape[1] - 1:
                output = output.rstrip()

            outfile.write(output)

            if x == self.shape[1] - 1:
                outfile.write("\n")

            it.iternext()
            
pp = pprint.PrettyPrinter(indent=2)

**An Introduction to Exploration: Epsilon-Greedy Exploration**

Exploration is a key concept in RL. In order to find the best policies, an agent needs to explore the environment. By exploring, the agent can experience new states and rewards. In the TD learning notebook, the agent explored GridWorld by taking a random action at every step. While random action explorations can work in some environments, the downside is the agent can spend too much time exploring bad states or states that have already been explored fully and not enough time exploring promising states. A simple--yet surprisingly effective--approach to exploration is Epsilon-Greedy exploration. A epsilon percentage of the time, the agent chooses a random action. The remaining amount of the time (1-epsilon) the agent choose the best estimated action aka the *greedy action*. Epsilon can be a fixed value between 0 and 1 or can start at a high value and gradually decay over time (ie start at .99 and decay to 0.01). In this notebook we will used a fixed epsilon value of 0.1. Below is a simple example of epsilon-greedy exploration.


In [4]:
#declare the environment
env = GridworldEnv()
#reset the environment and get the agent's current position (observation)
current_state = env.reset()
env._render()
print("")
action_dict = {0:"UP",1:"RIGHT", 2:"DOWN",3:"LEFT"}
greedy_dict = {0:3,1:3,2:3,3:3,
              4:0,5:0,6:0,7:0,
              8:2,9:2,10:2,11:2,
              12:1,13:1,14:1,15:1}
epsilon = 0.1

for i in range(10):
  #choose random action epsilon amount of the time
  if np.random.rand() < epsilon:
    action = env.action_space.sample()
    action_type = "random"
  else:
    #Choose a greedy action. We will learn greedy actions with Q learning in the following cells.
    action = greedy_dict[current_state]
    action_type = "greedy"
 
  current_state,reward,done,info = env.step(action)
  print("Agent took {} action {} and is now in state {} ".format(action_type, action_dict[action], current_state))
  env._render()
  print("")
  if done:
    print("Agent reached end of episode, resetting the env")
    print(env.reset())
    print("")
    env._render()
    print("")

T  o  o  o
o  o  x  o
o  o  o  o
o  o  o  T

Agent took greedy action UP and is now in state 2 
T  o  x  o
o  o  o  o
o  o  o  o
o  o  o  T

Agent took greedy action LEFT and is now in state 1 
T  x  o  o
o  o  o  o
o  o  o  o
o  o  o  T

Agent took greedy action LEFT and is now in state 0 
x  o  o  o
o  o  o  o
o  o  o  o
o  o  o  T

Agent reached end of episode, resetting the env
1

T  x  o  o
o  o  o  o
o  o  o  o
o  o  o  T

Agent took greedy action LEFT and is now in state 0 
x  o  o  o
o  o  o  o
o  o  o  o
o  o  o  T

Agent reached end of episode, resetting the env
10

T  o  o  o
o  o  o  o
o  o  x  o
o  o  o  T

Agent took greedy action LEFT and is now in state 9 
T  o  o  o
o  o  o  o
o  x  o  o
o  o  o  T

Agent took greedy action DOWN and is now in state 13 
T  o  o  o
o  o  o  o
o  o  o  o
o  x  o  T

Agent took greedy action RIGHT and is now in state 14 
T  o  o  o
o  o  o  o
o  o  o  o
o  o  x  T

Agent took greedy action RIGHT and is now in state 15 
T  o  o  o
o  o  o  

**The RL Training Loop**

In the next cell we are going to define the training loop and then run it in the following cell. The goal is to estimate the Q value of each state (the value of each state-action combination) using Q learning. q_value_array holds the estimated values. After each step the agent takes in the env, we update the q_value_array with the Q learning formula.

In [0]:
def q_learning_q_value_estimate(env,episodes=1000,alpha=0.05,discount_factor=1.0,epsilon=0.1):
  state_size = env.nS
  action_size = env.nA
  #initialize the estimated state values to zero
  q_value_array = np.zeros((state_size, action_size))
  #reset the env
  current_state = env.reset()
  #env._render()

  #run through each episode taking a random action each time
  #upgrade estimated state value after each action
  current_episode = 0
  while current_episode < episodes:
    #choose action based on epsilon-greedy policy
    if np.random.rand() < epsilon:
      eg_action = env.action_space.sample()
    else:
      #Choose a greedy action.
      eg_action = np.argmax(q_value_array[current_state])

    #take a step using epsilon-greedy action
    next_state, rew, done, info = env.step(eg_action)

    #Update Q values using Q learning update method
    max_q_value = np.max(q_value_array[next_state])
    q_value_array[current_state,eg_action] = q_value_array[current_state,eg_action] + \
      alpha * (rew + discount_factor*max_q_value - q_value_array[current_state,eg_action])

    #if the epsiode is done, reset the env, if not the next state becomes the current state and the loop repeats
    if done:
      current_state = env.reset()
      current_episode += 1
    else:
      current_state = next_state

  return q_value_array

In [7]:
#run episodes with Q learning and get the state value estimates
q_values = q_learning_q_value_estimate(env,episodes=10000,alpha=0.01)

print("All Q Value Estimates:")
print(np.round(q_values.reshape((16,4)),1))
print("each row is a state, each column is an action")
print("")

greedy_q_value_estimates = np.max(q_values,axis=1)
print("Greedy Q Value Estimates:")
print(np.round(greedy_q_value_estimates.reshape(env.shape),1))
print("estimate of the optimal State value at each state")
print("")

All Q Value Estimates:
[[ 0.   0.   0.   0. ]
 [-1.4 -1.8 -1.8 -1. ]
 [-2.1 -2.4 -2.2 -2. ]
 [-3.  -2.9 -2.9 -2.9]
 [-1.  -1.6 -1.7 -1.4]
 [-2.  -2.2 -2.2 -2. ]
 [-2.7 -2.7 -2.7 -2.7]
 [-2.3 -2.2 -2.  -2.2]
 [-2.  -2.2 -2.3 -2.2]
 [-2.7 -2.7 -2.7 -2.7]
 [-2.2 -2.  -2.  -2.1]
 [-1.8 -1.4 -1.  -1.8]
 [-2.9 -2.9 -2.9 -3. ]
 [-2.2 -2.  -2.1 -2.4]
 [-1.7 -1.  -1.4 -1.9]
 [ 0.   0.   0.   0. ]]
each row is a state, each column is an action

Greedy Q Value Estimates:
[[ 0.  -1.  -2.  -2.9]
 [-1.  -2.  -2.7 -2. ]
 [-2.  -2.7 -2.  -1. ]
 [-2.9 -2.  -1.   0. ]]
estimate of the optimal State value at each state



The first output shows the estimated value for each action in each state. Ie row 4 column 4 is the value if the agent was in the upper right grid cell and took that action left. In the second output, we take the best action for each of the 16 states and show the agent's estimate of the state value assuming the agent always acts greedily.